This will gather **ALL** founder sequence files based on <REDACTED> to a folder specified by you which will then be used for the other scripts, too.

You need a authenticated access to the spreadsheet from within the Jupyter session. It needs to open the browser, so I recommend getting the auth file on your local machine and copying it to the server.

do this locally with your email
````r
library("googlesheets4")
gs4_auth(email = "USER@REDACTED",cache="~/gcache")
````
then copy `~/gcache` (a folder) to the the server and proceed with your account below.

To effortlessly get the data from engram, you also need to enable a key-based authentification so that no password is needed to move files from engram to the cluster

````bash

# password-free key (should still be as save as the file is only on the password-protected cluster)
ssh-keygen -t rsa -P "" -f ~/.ssh/REDACTED
ssh-copy-id -i ~/.ssh/REDACTED USER@REDACTED

# now this should work w/o password
ssh -o "IdentitiesOnly=yes" -i ~/.ssh/engram USER@REDACTED

````

In [1]:
# set important data in the config file first!
source("config.R")
source("../../extra-R-functions.R")
# this user needs to have access to engram and google sheet 'REDACTED'

v Reading from Status of mouse sequencing.

v Range ''All founder files''.


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'data.table'


The following objects are masked from 'package:dplyr':

    between, first, last


This can lead to most dplyr functions being overwritten by tidytable functions.

This can lead to most tidyr functions being overwritten by tidytable functions.


Attaching package: 'tidytable'


The following objects are masked from 'package:data.table':

    between, first, fread, last


The following objects are masked from 'package:tidyr':

    complete, crossing, drop_na, expand, expand_grid, extract, fill,
    nest, nesting, pivot_longer, pivot_wider, replace_na, separate,
    separate_longer_delim, separate_rows, separate_wider_delim,
    separate_wider_regex, tribble, uncount, unite, unnest

In [2]:
# unifying whether folders end with "/"
files_df$Engram_folder[!endsWith(files_df$Engram_folder, "/")]=paste0(files_df$Engram_folder[!endsWith(files_df$Engram_folder, "/")], "/")

In [3]:
head(files_df)

File1,File2,ID,Engram_folder
<chr>,<chr>,<chr>,<chr>
F1_fwd_paired.fq.gz,F1_rev_paired.fq.gz,SW_1,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/
F2_fwd_paired.fq.gz,F2_rev_paired.fq.gz,SW_2,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/
F3_fwd_paired.fq.gz,F3_rev_paired.fq.gz,SW_3,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/
F4_fwd_paired.fq.gz,F4_rev_paired.fq.gz,SW_4,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/
F5_fwd_paired.fq.gz,F5_rev_paired.fq.gz,SW_5,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/
F6_fwd_paired.fq.gz,F6_rev_paired.fq.gz,SW_6,/home/jr3950/mnt/sequencing_data/201810xx/Founders-99592493/trimmed/


In [4]:
if(!dir.exists(folder_for_sequences)){
    dir.create(folder_for_sequences)
}

In [5]:
# lets check if maybe the trimmed version or final bam is already there!
# so that it is easy to add data / fix issues like I am doing right now...
files_df$skip=FALSE
for(i in 1:nrow(files_df)){
    ID=files_df$ID[i]
    ID_subset=files_df[files_df$ID==ID,]
    nthSequenceOfID=which(ID_subset$File1==files_df$File1[i])
    final_bam=paste0(bam_dir,"/",ID,".",nthSequenceOfID,".bam")
    trimmed_files_expected_1=paste0(trim_dir,"/",str_replace(str_replace(basename(files_df$File1[i]),fixed(".fq.gz"),
                                                ""),fixed(".fastq.gz"),""),"_val_1.fq.gz")
    trimmed_files_expected_2=paste0(trim_dir,"/",str_replace(str_replace(basename(files_df$File2[i]),fixed(".fq.gz"),
                                                ""),fixed(".fastq.gz"),""),"_val_2.fq.gz")
    if(file.exists(final_bam) | (file.exists(trimmed_files_expected_1)&file.exists(trimmed_files_expected_2))){
        files_df$skip[i]=TRUE
        
    }else{
        print(paste("missing",final_bam))
    }
}

[1] "missing /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_4.2.bam"
[1] "missing /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_31.1.bam"
[1] "missing /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_85.1.bam"
[1] "missing /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_87.1.bam"
[1] "missing /moto/ziab/users/jr3950/data/genomes/tmp_founders/bam/SW_1133.6.bam"


In [6]:
files_df[files_df$skip==FALSE,]

File1,File2,ID,Engram_folder,skip
<chr>,<chr>,<chr>,<chr>,<lgl>
18163FL-02-01-01_S1_L006_R1_001.fastq.gz,18163FL-02-01-01_S1_L006_R2_001.fastq.gz,SW_4,/home/jr3950/mnt/sequencing_data/20190124/18163-02/,FALSE
18163FL-02-01-04_S4_L006_R1_001.fastq.gz,18163FL-02-01-04_S4_L006_R2_001.fastq.gz,SW_31,/home/jr3950/mnt/sequencing_data/20190124/18163-02/,FALSE
18163FL-02-01-06_S6_L006_R1_001.fastq.gz,18163FL-02-01-06_S6_L006_R2_001.fastq.gz,SW_85,/home/jr3950/mnt/sequencing_data/20190124/18163-02/,FALSE
18163FL-02-01-07_S7_L006_R1_001.fastq.gz,18163FL-02-01-07_S7_L006_R2_001.fastq.gz,SW_87,/home/jr3950/mnt/sequencing_data/20190124/18163-02/,FALSE
18163FL-08-01-02_S2_R1_001.fastq.gz,18163FL-08-01-02_S2_R2_001.fastq.gz,SW_1133,/home/jr3950/mnt/sequencing_data/20191118/18163-08/,FALSE


In [7]:
if(sum(files_df$skip==FALSE)==0){
    stop("nothing to do")
    }

[1] 5

In [8]:
for(i in which(files_df$skip==FALSE)){
    cmd=paste0("rsync -avzPhe \"ssh -o 'IdentitiesOnly=yes' -i /moto/home/",columbia_username,"/.ssh/engram\" ",columbia_username,"@engram-xfer-01.rc.zi.columbia.edu:",files_df$Engram_folder[i],files_df$File1[i]," ",folder_for_sequences)
    print(system(command=cmd, intern=TRUE))
    cmd=paste0("rsync -avzPhe \"ssh -o 'IdentitiesOnly=yes' -i /moto/home/",columbia_username,"/.ssh/engram\" ",columbia_username,"@engram-xfer-01.rc.zi.columbia.edu:",files_df$Engram_folder[i],files_df$File2[i]," ",folder_for_sequences)
    print(system(command=cmd, intern=TRUE))
}

[1] "receiving incremental file list"                   
[2] ""                                                  
[3] "sent 11 bytes  received 88 bytes  198.00 bytes/sec"
[4] "total size is 4.04G  speedup is 40,774,271.39"     
[1] "receiving incremental file list"                  
[2] ""                                                 
[3] "sent 11 bytes  received 88 bytes  66.00 bytes/sec"
[4] "total size is 4.33G  speedup is 43,705,792.26"    
[1] "receiving incremental file list"                   
[2] ""                                                  
[3] "sent 11 bytes  received 88 bytes  198.00 bytes/sec"
[4] "total size is 4.44G  speedup is 44,803,048.62"     
[1] "receiving incremental file list"                  
[2] ""                                                 
[3] "sent 11 bytes  received 88 bytes  66.00 bytes/sec"
[4] "total size is 4.82G  speedup is 48,650,067.24"    
[1] "receiving incremental file list"                   
[2] ""                                 